<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

------
# **Backtesting definitivo RSI**
------

# 1. Bibliotecas

In [ ]:
%pip install --upgrade --no-cache-dir git+https://github.com/StreamAlpha/tvdatafeed.git

In [1]:
# Análise e manipulação de dados
import numpy as np
import pandas as pd

# Dados séries temporais
from datetime import datetime, timedelta
import pytz

# Fonte de dados
from tvDatafeed import TvDatafeed, Interval
import MetaTrader5 as mt5

# Indicadores e backtesting
import ta
import vectorbt as vbt

# Bibliotecas gráficas
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 2.Obtenção de dados

## 2.1. Cotações OHLC - TradingView - Week a 5min

In [2]:
lista_tickers = ['WDO1!', 'WIN1!', 'ABEV3', 'ALPA4', 'ALSO3', 'ARZZ3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4',
                'BBSE3', 'BEEF3', 'BPAC11', 'BPAN4', 'BRAP4', 'BRFS3', 'BRKM5', 'CASH3', 'CCRO3', 'CIEL3', 'CMIG4', 'CMIN3',
                'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'DXCO3', 'ECOR3', 'EGIE3', 'ELET3',
                'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3',
                'HYPE3', 'IGTI11', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3', 'LWSA3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3',
                'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PETZ3', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RAIZ4', 'RDOR3', 'RENT3',
                'RRRP3', 'SANB11', 'SBSP3', 'SLCE3', 'SMTO3', 'SOMA3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5',
                'VALE3', 'VBBR3', 'VIIA3', 'VIVT3', 'WEGE3', 'YDUQ3']

In [3]:
timeframes = [Interval.in_weekly, Interval.in_daily, Interval.in_1_hour,
                Interval.in_15_minute, Interval.in_5_minute]

In [4]:
cotacoes_tv_all = []

for ticker in lista_tickers:

    cotacoes_tv_ativo = []

    for timeframe in timeframes:

        cotacoes_tv_ativo_timef = TvDatafeed().get_hist(symbol=ticker, exchange='BMFBOVESPA', interval=timeframe, n_bars=10000)
        cotacoes_tv_ativo_timef['symbol'] = cotacoes_tv_ativo_timef['symbol'].str.replace('BMFBOVESPA:', '')
        cotacoes_tv_ativo_timef['timeframe'] = str(timeframe)[12:]

        cotacoes_tv_ativo.append(cotacoes_tv_ativo_timef)
    
    cotacoes_tv_all.append(cotacoes_tv_ativo)

you are using nologin method, data you access may be limited


In [5]:
list(enumerate(lista_tickers))[0:5]

[(0, 'WDO1!'), (1, 'WIN1!'), (2, 'ABEV3'), (3, 'ALPA4'), (4, 'ALSO3')]

In [6]:
timeframes_str = []
for i in timeframes:
    timeframes_str.append(str(i)[12:])
    
list(enumerate(timeframes_str))

[(0, 'weekly'), (1, 'daily'), (2, '1_hour'), (3, '15_minute'), (4, '5_minute')]

In [7]:
cotacoes_tv_all[1][3]

,symbol,open,high,low,close,volume,timeframe
datetime,,,,,,,
2022-09-01 09:00:00,WIN1!,111250.0,111415.0,110825.0,111185.0,618107.0,15_minute
2022-09-01 09:15:00,WIN1!,111185.0,111505.0,111150.0,111350.0,359003.0,15_minute
2022-09-01 09:30:00,WIN1!,111350.0,111495.0,111210.0,111245.0,485551.0,15_minute
2022-09-01 09:45:00,WIN1!,111245.0,111390.0,111020.0,111270.0,523212.0,15_minute
2022-09-01 10:00:00,WIN1!,111270.0,111485.0,110935.0,111175.0,971894.0,15_minute
...,...,...,...,...,...,...,...
2023-03-30 09:00:00,WIN1!,103680.0,104030.0,103570.0,103730.0,476896.0,15_minute
2023-03-30 09:15:00,WIN1!,103735.0,103950.0,103560.0,103790.0,404150.0,15_minute
2023-03-30 09:30:00,WIN1!,103795.0,104200.0,103745.0,103985.0,616797.0,15_minute


## 2.2. Cotações OHLC - TradingView - Monthly a Diario

In [2]:
lista_tickers = ['WDO1!', 'WIN1!', 'ABEV3', 'ALPA4', 'ALSO3', 'ARZZ3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4',
                'BBSE3', 'BEEF3', 'BPAC11', 'BPAN4', 'BRAP4', 'BRFS3', 'BRKM5', 'CASH3', 'CCRO3', 'CIEL3', 'CMIG4', 'CMIN3',
                'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'DXCO3', 'ECOR3', 'EGIE3', 'ELET3',
                'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3',
                'HYPE3', 'IGTI11', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3', 'LWSA3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3',
                'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PETZ3', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RAIZ4', 'RDOR3', 'RENT3',
                'RRRP3', 'SANB11', 'SBSP3', 'SLCE3', 'SMTO3', 'SOMA3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5',
                'VALE3', 'VBBR3', 'VIIA3', 'VIVT3', 'WEGE3', 'YDUQ3']

In [3]:
timeframes = [Interval.in_monthly, Interval.in_weekly, Interval.in_daily]

In [59]:
cotacoes_tv_all = []

for ticker in lista_tickers:

    cotacoes_tv_ativo = []

    for timeframe in timeframes:

        cotacoes_tv_ativo_timef = TvDatafeed().get_hist(symbol=ticker, exchange='BMFBOVESPA', interval=timeframe, n_bars=10000)
        cotacoes_tv_ativo_timef['symbol'] = cotacoes_tv_ativo_timef['symbol'].str.replace('BMFBOVESPA:', '')
        cotacoes_tv_ativo_timef['timeframe'] = str(timeframe)[12:]

        cotacoes_tv_ativo.append(cotacoes_tv_ativo_timef)
    
    cotacoes_tv_all.append(cotacoes_tv_ativo)

In [16]:
list(enumerate(lista_tickers))

[(0, 'WDO1!'),
 (1, 'WIN1!'),
 (2, 'ABEV3'),
 (3, 'ALPA4'),
 (4, 'ALSO3'),
 (5, 'ARZZ3'),
 (6, 'ASAI3'),
 (7, 'AZUL4'),
 (8, 'B3SA3'),
 (9, 'BBAS3'),
 (10, 'BBDC3'),
 (11, 'BBDC4'),
 (12, 'BBSE3'),
 (13, 'BEEF3'),
 (14, 'BPAC11'),
 (15, 'BPAN4'),
 (16, 'BRAP4'),
 (17, 'BRFS3'),
 (18, 'BRKM5'),
 (19, 'CASH3'),
 (20, 'CCRO3'),
 (21, 'CIEL3'),
 (22, 'CMIG4'),
 (23, 'CMIN3'),
 (24, 'COGN3'),
 (25, 'CPFE3'),
 (26, 'CPLE6'),
 (27, 'CRFB3'),
 (28, 'CSAN3'),
 (29, 'CSNA3'),
 (30, 'CVCB3'),
 (31, 'CYRE3'),
 (32, 'DXCO3'),
 (33, 'ECOR3'),
 (34, 'EGIE3'),
 (35, 'ELET3'),
 (36, 'ELET6'),
 (37, 'EMBR3'),
 (38, 'ENBR3'),
 (39, 'ENEV3'),
 (40, 'ENGI11'),
 (41, 'EQTL3'),
 (42, 'EZTC3'),
 (43, 'FLRY3'),
 (44, 'GGBR4'),
 (45, 'GOAU4'),
 (46, 'GOLL4'),
 (47, 'HAPV3'),
 (48, 'HYPE3'),
 (49, 'IGTI11'),
 (50, 'ITSA4'),
 (51, 'ITUB4'),
 (52, 'JBSS3'),
 (53, 'KLBN11'),
 (54, 'LREN3'),
 (55, 'LWSA3'),
 (56, 'MGLU3'),
 (57, 'MRFG3'),
 (58, 'MRVE3'),
 (59, 'MULT3'),
 (60, 'NTCO3'),
 (61, 'PCAR3'),
 (62, 'PETR3')

## 2.3. Cotações OHLC - MetaTrader

In [ ]:
mt5.initialize()

In [ ]:
lista_tickers = ['WDO$', 'WIN$', 'ABEV3', 'ALPA4', 'ALSO3', 'ARZZ3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4',
                'BBSE3', 'BEEF3', 'BPAC11', 'BPAN4', 'BRAP4', 'BRFS3', 'BRKM5', 'CASH3', 'CCRO3', 'CIEL3', 'CMIG4', 'CMIN3',
                'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'DXCO3', 'ECOR3', 'EGIE3', 'ELET3',
                'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3',
                'HYPE3', 'IGTI11', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3', 'LWSA3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3',
                'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PETZ3', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RAIZ4', 'RDOR3', 'RENT3',
                'RRRP3', 'SANB11', 'SBSP3', 'SLCE3', 'SMTO3', 'SOMA3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5',
                'VALE3', 'VBBR3', 'VIIA3', 'VIVT3', 'WEGE3', 'YDUQ3']

In [ ]:
for i in lista_tickers:
    selected=mt5.symbol_select(i, True)
    if not selected:
        print(i,"não localizado")
timeframes = [32769, 16408, 16385, 15, 5]
timezone = pytz.timezone("America/Sao_Paulo")
data_inicio = datetime(2013, 1, 1)
data_fim = datetime(2022, 12, 31)
data_inicio
timezone = pytz.timezone("America/Sao_Paulo")
data_inicio = datetime.now() - timedelta(days=10000)
data_fim = datetime.now()

cotacoes_mt5_all = []

for ticker in lista_tickers:

    cotacoes_mt5_ativo = []

    for timeframe in timeframes:

        cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, timeframe, data_inicio, data_fim)
        cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
        cotacoes_mt5_ticker['ticker'] = str(ticker)

        if timeframe==32769:
            cotacoes_mt5_ticker['timeframe'] = '1W'
        elif timeframe==16408:
            cotacoes_mt5_ticker['timeframe'] = '1D'
        elif timeframe==16385:
            cotacoes_mt5_ticker['timeframe'] = '1H'
        else:
            cotacoes_mt5_ticker['timeframe'] = str(timeframe)+'M'
        
        cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
        cotacoes_mt5_ticker = cotacoes_mt5_ticker.drop('time', axis=1)
        cotacoes_mt5_ativo.append(cotacoes_mt5_ticker)
    
    cotacoes_mt5_all.append(cotacoes_mt5_ativo)
list(enumerate(lista_tickers))[0:5]
timeframes_str = ['1W', '1D', '1H', '15M', '5M']
list(enumerate(timeframes_str))
cotacoes_mt5_all[2][0]

## 2.4. Cálculo indicadores

In [60]:
periodo_indicadores = 14

In [61]:
for i in range(len(cotacoes_tv_all)):
    for j in range(len(cotacoes_tv_ativo)):
        cotacoes_tv_all[i][j]['RSI'] = ta.momentum.RSIIndicator(cotacoes_tv_all[i][j].close, window=periodo_indicadores).rsi()
        #cotacoes_tv_all[i][j]['RSI_vbt'] = vbt.RSI.run(cotacoes_tv_all[i][j].close, window=periodo_indicadores).rsi
        #cotacoes_tv_all[i][j] = cotacoes_tv_all[i][j].iloc[15:]

# 3. Backtesting

In [62]:
for i in range(len(cotacoes_tv_all)):

    for j in range(len(cotacoes_tv_ativo)):

        cotacoes_tv_all[i][j]['entrada'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_above(30)
        cotacoes_tv_all[i][j]['saida'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_above(70)
        #cotacoes_tv_all[i][j]['short_entrada'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_below(70)
        #cotacoes_tv_all[i][j]['short_saida'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_below(30)

In [63]:
ls_backtests_all = []

for i in range(len(cotacoes_tv_all)):

    ls_backtests_ativo = []

    for j in range(len(cotacoes_tv_ativo)):

        backtest_ativo = vbt.Portfolio.from_signals(cotacoes_tv_all[i][j].close, entries=cotacoes_tv_all[i][j].entrada, exits=cotacoes_tv_all[i][j].saida,
                                                    #short_entries=cotacoes_tv_all[i][j].short_entrada, short_exits=cotacoes_tv_all[i][j].short_saida,
                                                    fees=0.00005,
                                                    size_type='Value', size=1,
                                                    )
        ls_backtests_ativo.append(backtest_ativo)

    ls_backtests_all.append(ls_backtests_ativo)

In [64]:
ls_ativos = []
ls_timeframes = []
ls_retorno = []
ls_n_trades = []
ls_annualized_return = []
ls_min_pnl = []


for i in range(len(ls_backtests_all)):

    for j in range(len(ls_backtests_ativo)):

        ls_ativos.append(i)
        ls_timeframes.append(j)
        
        ls_n_trades.append(ls_backtests_all[i][j].trades.count())

        ls_annualized_return = []

        lista_pnl = list(ls_backtests_all[i][j].trades.pnl.values)
        cash_init = [100]
        if lista_pnl == []:
            ls_min_pnl.append(1)
        else:
            ls_min_pnl.append(min(lista_pnl))
        for index,pnl in enumerate(lista_pnl):
            if pnl<=-1:
                pnl=0
                cash_init.append(cash_init[index]*pnl)
            else:
                cash_init.append(cash_init[index]*(1+pnl))
        if min(cash_init)==0:
            ls_retorno.append(0)
        else:
            ls_retorno.append(cash_init[-1])

In [65]:
from datetime import datetime as dt
import time

def toYearFraction(date):
    def sinceEpoch(date): # returns seconds since epoch
        return time.mktime(date.timetuple())
    s = sinceEpoch

    year = date.year
    startOfThisYear = dt(year=year, month=1, day=1)
    startOfNextYear = dt(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration

    return date.year + fraction

In [67]:
ls_ativos = []
ls_timeframes = []
ls_data_init = []
ls_year_init = []
ls_n_candles = []

for i in range(len(cotacoes_tv_all)):

    for j in range(len(cotacoes_tv_ativo)):

        ls_ativos.append(i)
        ls_timeframes.append(j)
        ls_data_init.append(cotacoes_tv_all[i][j].index[0])
        ls_year_init.append(toYearFraction(cotacoes_tv_all[i][j].index[0]))
        ls_n_candles.append(len(cotacoes_tv_all[i][j]))

In [68]:
df_result_01 = pd.DataFrame([ls_ativos, ls_timeframes, ls_retorno, ls_n_trades, ls_min_pnl, ls_data_init, ls_year_init, ls_n_candles], \
                            index=['Ativo', 'Timeframe', 'Cash_final', 'N_trades', 'Min_PnL', 'Data_init', 'Ano_init','N_candles']).transpose()
df_result_01

,Ativo,Timeframe,Cash_final,N_trades,Min_PnL,Data_init,Ano_init,N_candles
0,0,0,100,0,1,2011-10-03 09:00:00,2011.754452,138
1,0,1,105.2764,1,0.052764,2011-10-03 09:00:00,2011.754452,600
2,0,2,156.111165,11,-0.084366,2011-10-07 09:00:00,2011.765411,2842
3,1,0,100,0,1,2005-03-01 09:00:00,2005.162671,217
4,1,1,385.631907,4,0.091303,2005-02-28 09:00:00,2005.159932,940
...,...,...,...,...,...,...,...,...
265,88,1,294.933364,3,0.370252,2000-01-03 09:00:00,2000.006489,1213
266,88,2,648.535766,18,-0.316787,2000-01-03 09:00:00,2000.006489,5544
267,89,0,100,0,1,2007-07-02 10:00:00,2007.499772,189
268,89,1,66.252545,5,-0.731862,2007-07-30 10:00:00,2007.576484,818


In [69]:
#result_final = pd.pivot_table(df_result_01, values="Cash_final", index=["Ativo"], columns=["Timeframe"], aggfunc='mean')
#result_final.style.format('{:.2f}', na_rep="").applymap(lambda x: 'background-color : #d40202' if x>100 else 'background-color : #ffe6e6')

In [70]:
df_result_01['Tempo_em_anos'] = toYearFraction(dt.now()) - df_result_01.Ano_init
df_result_01['Retorno_annualizado']  = (df_result_01.Cash_final/100)**(1/df_result_01.Tempo_em_anos) - 1
df_result_01

,Ativo,Timeframe,Cash_final,N_trades,Min_PnL,Data_init,Ano_init,N_candles,Tempo_em_anos,Retorno_annualizado
0,0,0,100,0,1,2011-10-03 09:00:00,2011.754452,138,11.490615,0.0
1,0,1,105.2764,1,0.052764,2011-10-03 09:00:00,2011.754452,600,11.490615,0.004485
2,0,2,156.111165,11,-0.084366,2011-10-07 09:00:00,2011.765411,2842,11.479656,0.039561
3,1,0,100,0,1,2005-03-01 09:00:00,2005.162671,217,18.082396,0.0
4,1,1,385.631907,4,0.091303,2005-02-28 09:00:00,2005.159932,940,18.085136,0.077487
...,...,...,...,...,...,...,...,...,...,...
265,88,1,294.933364,3,0.370252,2000-01-03 09:00:00,2000.006489,1213,23.238578,0.047643
266,88,2,648.535766,18,-0.316787,2000-01-03 09:00:00,2000.006489,5544,23.238578,0.083775
267,89,0,100,0,1,2007-07-02 10:00:00,2007.499772,189,15.745296,0.0
268,89,1,66.252545,5,-0.731862,2007-07-30 10:00:00,2007.576484,818,15.668583,-0.025933


In [71]:
result_final = pd.pivot_table(df_result_01, values="Retorno_annualizado", index=["Ativo"], columns=["Timeframe"], aggfunc='mean')
result_final = round(result_final*100,1)
result_final.index = lista_tickers

timeframes_reduzido = [str(i)[12:] for i in timeframes]
result_final.columns = timeframes_reduzido

In [72]:
result_final[result_final.monthly>0]['monthly'].describe()

count    31.000000
mean      6.116129
std       3.484547
min       0.900000
25%       4.000000
50%       5.200000
75%       7.650000
max      14.700000
Name: monthly, dtype: float64

In [73]:
result_final[result_final.weekly>0]['weekly'].describe()

count    61.000000
mean      5.831148
std       3.755287
min       0.100000
25%       3.000000
50%       5.500000
75%       7.700000
max      20.200000
Name: weekly, dtype: float64

In [74]:
result_final[result_final.daily>0]['daily'].describe()

count    55.000000
mean      6.958182
std       6.420819
min       0.300000
25%       3.400000
50%       5.600000
75%       8.450000
max      33.000000
Name: daily, dtype: float64

In [82]:
df_result_01.N_trades.sort_values().head(50)

0      0
148    0
147    0
144    0
141    0
129    0
120    0
114    0
102    0
96     0
93     0
255    0
84     0
81     0
75     0
66     0
57     0
258    0
150    0
42     0
243    0
156    0
232    0
231    0
235    0
225    0
237    0
222    0
219    0
217    0
216    0
207    0
201    0
189    0
186    0
240    0
165    0
160    0
159    0
153    0
261    0
234    0
36     0
15     0
3      0
264    0
267    0
6      0
18     0
33     0
Name: N_trades, dtype: object

In [78]:
result_final.style.format('{:.2f}', na_rep="").applymap(lambda x: "font-weight: ; color: white; background-color : #CD0000" if x == -100 \
                                                        else ( "font-weight: ; color: black; background-color : #FF6666" if x <0 \
                                                              else ("font-weight: ; color: black; background-color : #C1FFC1" if x <10 \
                                                                  else ("font-weight: ; color: black; background-color : #43CD80" if x <20 \
                                                                        else "font-weight: ; color: white; background-color : forestgreen"))))

,monthly,weekly,daily
WDO1!,0.00,0.40,4.00
WIN1!,0.00,7.70,1.30
ABEV3,0.00,4.80,0.30
ALPA4,12.20,10.00,-2.20
ALSO3,4.30,-3.30,-3.90
ARZZ3,0.00,8.50,7.40
ASAI3,0.00,13.80,-5.40
AZUL4,0.00,8.90,-0.80
B3SA3,6.90,0.70,-1.30
BBAS3,0.00,8.90,6.10


In [47]:
df_result_01[df_result_01.Ativo==7]

,Ativo,Timeframe,Cash_final,N_trades,Min_PnL,Data_init,Ano_init,N_candles,Tempo_em_anos,Retorno_annualizado
21,7,0,174.262515,1,0.742625,2017-04-03 10:00:00,2017.253196,72,5.991804,0.097124
22,7,1,374.865864,6,-0.573289,2017-04-10 10:00:00,2017.272374,312,5.972626,0.247626
23,7,2,28.966305,11,-0.564809,2017-04-11 10:00:00,2017.275114,1479,5.969886,-0.187426


In [50]:
ls_backtests_all[7][1].plot_trade_pnl()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'customdata': array([[0.        , 0.        , 0.26943405, 0.26943405],
                                   [1.        , 1.        , 0.60170057, 0.60170057],
                                   [2.        , 2.        , 0.28863068, 0.28863068],
                                   [3.        , 3.        , 1.41362241, 1.41362241],
                                   [4.        , 4.        , 0.38916307, 0.38916307]]),
              'hovertemplate': ('Exit Trade Id: %{customdata[0]' ... 'r>Return: %{customdata[3]:.2%}'),
              'marker': {'color': '#37B13F',
                         'line': {'color': 'rgb(38,123,44)', 'width': 1},
                         'opacity': array([0.75      , 0.79355924, 0.75251663, 0.9       , 0.76569615]),
                         'size': array([ 7.        ,  9.03276465,  7.11744258, 14.        ,  7.73248702]),
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Closed - Profit',
              'type': 'scatter',
              'uid': '7492b073-ee36-4247-918d-2b09c424dfe0',
              'x': array([datetime.datetime(2018, 6, 25, 10, 0),
                          datetime.datetime(2018, 11, 12, 9, 0),
                          datetime.datetime(2020, 3, 9, 10, 0),
                          datetime.datetime(2020, 11, 30, 10, 0),
                          datetime.datetime(2021, 10, 25, 10, 0)], dtype=object),
              'y': array([0.26943405, 0.60170057, 0.28863068, 1.41362241, 0.38916307])},
             {'customdata': array([[ 5.        ,  5.        , -0.57328944, -0.57328944]]),
              'hovertemplate': ('Exit Trade Id: %{customdata[0]' ... 'r>Return: %{customdata[3]:.2%}'),
              'marker': {'color': '#FFAA00',
                         'line': {'color': 'rgb(178,118,0)', 'width': 1},
                         'opacity': array([0.78983462]),
                         'size': array([8.85894891]),
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Open',
              'type': 'scatter',
              'uid': 'd6a50756-fa0f-4067-a105-48fe00f4264a',
              'x': array([datetime.datetime(2023, 3, 27, 10, 0)], dtype=object),
              'y': array([-0.57328944])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'shapes': [{'line': {'color': 'gray', 'dash': 'dash'},
                           'type': 'line',
                           'x0': 0,
                           'x1': 1,
                           'xref': 'paper',
                           'y0': 0,
                           'y1': 0,
                           'yref': 'y'}],
               'template': '...',
               'width': 700,
               'yaxis': {'tickformat': '.2%'}}
})

In [27]:
i = 7
j = 1
ls_ativos = []
ls_timeframes = []
ls_retorno = []
ls_n_trades = []
ls_annualized_return = []
ls_min_pnl = []

In [51]:

ls_ativos = []
ls_timeframes = []
ls_retorno = []
ls_n_trades = []
ls_annualized_return = []
ls_min_pnl = []


for i in range(len(ls_backtests_all)):

    for j in range(len(ls_backtests_ativo)):

        ls_ativos.append(i)
        ls_timeframes.append(j)
        
        ls_n_trades.append(ls_backtests_all[i][j].trades.count())

        ls_annualized_return = []

        lista_pnl = list(ls_backtests_all[i][j].trades.pnl.values)
        cash_init = [100]
        if lista_pnl == []:
            ls_min_pnl.append(1)
        else:
            ls_min_pnl.append(min(lista_pnl))
        for index,pnl in enumerate(lista_pnl):
            if pnl<=-1:
                pnl=0
                cash_init.append(cash_init[index]*pnl)
            else:
                cash_init.append(cash_init[index]*(1+pnl))
        if min(cash_init)==0:
            ls_retorno.append(0)
        else:
            ls_retorno.append(cash_init[-1])

In [56]:
ls_backtests_all[7][1].stats()

c:\Users\jamir\AppData\Local\Programs\Python\Python310\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\jamir\AppData\Local\Programs\Python\Python310\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\jamir\AppData\Local\Programs\Python\Python310\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\jamir\AppData\Local\Programs\Python\Python310\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2017-04-10 10:00:00
End                           2023-03-27 10:00:00
Period                                        312
Start Value                                 100.0
End Value                              102.389261
Total Return [%]                         2.389261
Benchmark Return [%]                   -47.281427
Max Gross Exposure [%]                   2.072857
Total Fees Paid                          0.000603
Max Drawdown [%]                         0.855273
Max Drawdown Duration                        72.0
Total Trades                                    6
Total Closed Trades                             5
Total Open Trades                               1
Open Trade PnL                          -0.573289
Win Rate [%]                                100.0
Best Trade [%]                         141.362241
Worst Trade [%]                         26.943405
Avg Winning Trade [%]                   59.251016
Avg Losing Trade [%]                          NaN


In [57]:
ls_backtests_all[7][1].trades.pnl.values

array([ 0.26943405,  0.60170057,  0.28863068,  1.41362241,  0.38916307,
       -0.57328944])

In [58]:
cash_init

[100,
 62.55238971479539,
 71.23654060113513,
 43.65751424710615,
 25.038652868004696,
 30.112791463024838,
 26.59247211461808,
 29.66647637427573,
 39.4824053273471,
 46.13503882179273,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
ls_backtests_all[7][1]

In [134]:
print('média',result_final.mean())
print('mediana',result_final.median())

média monthly   -51.084444
weekly    -64.078889
daily     -66.394444
dtype: float64
mediana monthly   -100.0
weekly    -100.0
daily     -100.0
dtype: float64


In [135]:
result_final.describe()

,monthly,weekly,daily
count,90.000000,90.000000,90.000000
mean,-51.084444,-64.078889,-66.394444
std,51.833983,48.403360,48.931352
min,-100.000000,-100.000000,-100.000000
25%,-100.000000,-100.000000,-100.000000
50%,-100.000000,-100.000000,-100.000000
75%,0.225000,-5.000000,-13.800000
max,20.600000,28.600000,77.900000


# 4. Backestinf incluindo DAYTRADE

In [73]:
for i in range(len(cotacoes_tv_all)):

    for j in range(len(cotacoes_tv_ativo)):

        if (j==0) or (j==1) or (j==2):
            cotacoes_tv_all[i][j]['entrada'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_above(30)
            cotacoes_tv_all[i][j]['saida'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_above(70)
            cotacoes_tv_all[i][j]['short_entrada'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_below(70)
            cotacoes_tv_all[i][j]['short_saida'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_below(30)

        else:
            cotacoes_tv_all[i][j]['entrada'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_above(30).vbt.signals.AND(cotacoes_tv_all[i][j].index.hour<15)
            cotacoes_tv_all[i][j]['saida'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_above(70).vbt.signals.OR(cotacoes_tv_all[i][j].index.hour>16)
            cotacoes_tv_all[i][j]['short_entrada'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_below(70).vbt.signals.AND(cotacoes_tv_all[i][j].index.hour<15)
            cotacoes_tv_all[i][j]['short_saida'] = cotacoes_tv_all[i][j]['RSI'].vbt.crossed_below(30).vbt.signals.OR(cotacoes_tv_all[i][j].index.hour>16)

In [74]:
ls_backtests_all = []

for i in range(len(cotacoes_tv_all)):

    ls_backtests_ativo = []

    for j in range(len(cotacoes_tv_ativo)):

        backtest_ativo = vbt.Portfolio.from_signals(cotacoes_tv_all[i][j].close, entries=cotacoes_tv_all[i][j].entrada, exits=cotacoes_tv_all[i][j].saida,
                                                    short_entries=cotacoes_tv_all[i][j].short_entrada, short_exits=cotacoes_tv_all[i][j].short_saida,
                                                    fees=0.00005,
                                                    size_type='Value', size=1,
                                                    )
        ls_backtests_ativo.append(backtest_ativo)

    ls_backtests_all.append(ls_backtests_ativo)

ls_ativos = []
ls_timeframes = []
ls_retorno = []
ls_n_trades = []
ls_annualized_return = []
ls_min_pnl = []


for i in range(len(ls_backtests_all)):

    for j in range(len(ls_backtests_ativo)):

        ls_ativos.append(i)
        ls_timeframes.append(j)
        
        ls_n_trades.append(ls_backtests_all[i][j].trades.count())

        ls_annualized_return = []

        lista_pnl = list(ls_backtests_all[i][j].trades.pnl.values)
        cash_init = [100]
        if lista_pnl == []:
            ls_min_pnl.append(1)
        else:
            ls_min_pnl.append(min(lista_pnl))
        for index,pnl in enumerate(lista_pnl):
            if pnl<=-1:
                pnl=0
                cash_init.append(cash_init[index]*pnl)
            else:
                cash_init.append(cash_init[index]*(1+pnl))
        if min(cash_init)==0:
            ls_retorno.append(0)
        else:
            ls_retorno.append(cash_init[-1])

In [75]:
from datetime import datetime as dt
import time

def toYearFraction(date):
    def sinceEpoch(date): # returns seconds since epoch
        return time.mktime(date.timetuple())
    s = sinceEpoch

    year = date.year
    startOfThisYear = dt(year=year, month=1, day=1)
    startOfNextYear = dt(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration

    return date.year + fraction

In [76]:
ls_ativos = []
ls_timeframes = []
ls_data_init = []
ls_year_init = []
ls_n_candles = []

for i in range(len(cotacoes_tv_all)):

    for j in range(len(cotacoes_tv_ativo)):

        ls_ativos.append(i)
        ls_timeframes.append(j)
        ls_data_init.append(cotacoes_tv_all[i][j].index[0])
        ls_year_init.append(toYearFraction(cotacoes_tv_all[i][j].index[0]))
        ls_n_candles.append(len(cotacoes_tv_all[i][j]))

df_result_01 = pd.DataFrame([ls_ativos, ls_timeframes, ls_retorno, ls_n_trades, ls_min_pnl, ls_data_init, ls_year_init, ls_n_candles], \
                            index=['Ativo', 'Timeframe', 'Cash_final', 'N_trades', 'Min_PnL', 'Data_init', 'Ano_init','N_candles']).transpose()
df_result_01
#df_result_01.sort_values('Cash_final')

df_result_01['Tempo_em_anos'] = toYearFraction(dt.now()) - df_result_01.Ano_init
df_result_01['Retorno_annualizado']  = (df_result_01.Cash_final/100)**(1/df_result_01.Tempo_em_anos) - 1
df_result_01

,Ativo,Timeframe,Cash_final,N_trades,Min_PnL,Data_init,Ano_init,N_candles,Tempo_em_anos,Retorno_annualizado
0,0,0,0,2,-1.380214,2011-10-03 09:00:00,2011.754452,600,11.487875,-1.0
1,0,1,51.960512,22,-0.569966,2011-10-07 09:00:00,2011.765411,2841,11.476916,-0.055447
2,0,2,120.789765,52,-0.105447,2021-01-04 09:00:00,2021.009247,5243,2.233081,0.088264
3,0,3,101.839946,78,-0.020617,2022-09-01 09:00:00,2022.666781,5311,0.575547,0.032185
4,0,4,102.926932,55,-0.012144,2023-01-23 09:00:00,2023.061301,5128,0.181026,0.172766
...,...,...,...,...,...,...,...,...,...,...
445,89,0,0,9,-1.143987,2007-07-30 10:00:00,2007.576484,818,15.665843,-1.0
446,89,1,0,29,-1.197503,2007-07-30 10:00:00,2007.576484,3870,15.665843,-1.0
447,89,2,74.382269,59,-0.414742,2020-01-02 10:00:00,2020.003871,6442,3.238457,-0.087335
448,89,3,78.68888,86,-0.121807,2022-08-01 10:00:00,2022.581963,5204,0.660364,-0.304367


In [77]:
result_final_DT = pd.pivot_table(df_result_01, values="Cash_final", index=["Ativo"], columns=["Timeframe"], aggfunc='mean')
result_final_DT.style.format('{:.2f}', na_rep="").applymap(lambda x: 'background-color : #e6ffe6' if x>0.05 else 'background-color : #ffe6e6')

Timeframe,0,1,2,3,4
Ativo,,,,,
0,0.00,51.96,120.79,101.84,102.93
1,225.01,24.86,133.08,91.40,100.00
2,0.00,0.00,61.48,97.39,102.43
3,0.00,0.00,75.32,66.65,107.84
4,54.26,0.00,81.08,68.75,95.09
5,172.36,61.15,59.09,86.57,119.10
6,172.87,74.89,202.35,117.57,102.06
7,361.26,29.93,6.66,80.93,99.65
8,0.00,0.24,68.48,75.09,103.68


In [78]:
result_final_DT = pd.pivot_table(df_result_01, values="Retorno_annualizado", index=["Ativo"], columns=["Timeframe"], aggfunc='mean')
result_final_DT.style.format('{:.2f}', na_rep="").applymap(lambda x: 'background-color : #e6ffe6' if x>0.05 else 'background-color : #ffe6e6')

Timeframe,0,1,2,3,4
Ativo,,,,,
0,-1.00,-0.06,0.09,0.03,0.17
1,0.05,-0.07,0.14,-0.14,-0.00
2,-1.00,-1.00,-0.14,-0.04,0.12
3,-1.00,-1.00,-0.08,-0.46,0.41
4,-0.05,-1.00,-0.06,-0.43,-0.21
5,0.05,-0.04,-0.15,-0.20,1.22
6,0.30,-0.13,0.40,0.28,0.10
7,0.24,-0.18,-0.57,-0.27,-0.02
8,-1.00,-0.32,-0.11,-0.35,0.18


In [61]:
ls_backtests_all[13][4].plot_trade_pnl()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'customdata': array([[0.00000000e+00, 0.00000000e+00, 7.32979198e-03, 7.32979198e-03],
                                   [1.00000000e+00, 1.00000000e+00, 5.13515660e-02, 5.13515660e-02],
                                   [3.00000000e+00, 3.00000000e+00, 1.66215719e-02, 1.66215719e-02],
                                   [4.00000000e+00, 4.00000000e+00, 2.10092348e-02, 2.10092348e-02],
                                   [5.00000000e+00, 5.00000000e+00, 4.49852288e-02, 4.49852288e-02],
                                   [6.00000000e+00, 6.00000000e+00, 1.53646907e-02, 1.53646907e-02],
                                   [7.00000000e+00, 7.00000000e+00, 1.09960509e-02, 1.09960509e-02],
                                   [8.00000000e+00, 8.00000000e+00, 1.41125969e-02, 1.41125969e-02],
                                   [1.00000000e+01, 1.00000000e+01, 3.17741036e-02, 3.17741036e-02],
                                   [1.10000000e+01, 1.10000000e+01, 1.22450617e-02, 1.22450617e-02],
                                   [1.40000000e+01, 1.40000000e+01, 2.67978966e-02, 2.67978966e-02],
                                   [1.60000000e+01, 1.60000000e+01, 3.33536299e-02, 3.33536299e-02],
                                   [1.70000000e+01, 1.70000000e+01, 2.83886848e-03, 2.83886848e-03],
                                   [1.80000000e+01, 1.80000000e+01, 1.01719736e-02, 1.01719736e-02],
                                   [1.90000000e+01, 1.90000000e+01, 1.54317678e-02, 1.54317678e-02],
                                   [2.00000000e+01, 2.00000000e+01, 3.27483577e-02, 3.27483577e-02],
                                   [2.10000000e+01, 2.10000000e+01, 2.11593774e-02, 2.11593774e-02],
                                   [2.20000000e+01, 2.20000000e+01, 3.71319434e-02, 3.71319434e-02],
                                   [2.30000000e+01, 2.30000000e+01, 2.23100077e-02, 2.23100077e-02],
                                   [2.40000000e+01, 2.40000000e+01, 1.82495413e-02, 1.82495413e-02],
                                   [2.50000000e+01, 2.50000000e+01, 7.68777259e-03, 7.68777259e-03],
                                   [2.60000000e+01, 2.60000000e+01, 1.53567233e-02, 1.53567233e-02],
                                   [2.70000000e+01, 2.70000000e+01, 1.27096077e-02, 1.27096077e-02],
                                   [2.80000000e+01, 2.80000000e+01, 1.97422619e-02, 1.97422619e-02],
                                   [3.00000000e+01, 3.00000000e+01, 2.83007315e-02, 2.83007315e-02],
                                   [3.10000000e+01, 3.10000000e+01, 3.81494536e-02, 3.81494536e-02],
                                   [3.20000000e+01, 3.20000000e+01, 5.12674216e-03, 5.12674216e-03],
                                   [3.30000000e+01, 3.30000000e+01, 6.94190141e-03, 6.94190141e-03],
                                   [3.50000000e+01, 3.50000000e+01, 4.51581981e-02, 4.51581981e-02],
                                   [3.60000000e+01, 3.60000000e+01, 1.80377576e-02, 1.80377576e-02],
                                   [3.70000000e+01, 3.70000000e+01, 1.75313602e-02, 1.75313602e-02],
                                   [3.80000000e+01, 3.80000000e+01, 2.30607940e-02, 2.30607940e-02],
                                   [4.00000000e+01, 4.00000000e+01, 2.15086863e-02, 2.15086863e-02],
                                   [4.10000000e+01, 4.10000000e+01, 2.02529646e-02, 2.02529646e-02],
                                   [4.20000000e+01, 4.20000000e+01, 1.30701054e-02, 1.30701054e-02],
                                   [4.30000000e+01, 4.30000000e+01, 4.87430284e-02, 4.87430284e-02],
                                   [4.40000000e+01, 4.40000000e+01, 3.50044949e-02, 3.50044949e-02],
                                   [4.50000000e+01, 4.50000000e+01, 3.80186613e-02, 3.80186613e-02],
                                   [4.60000000e+01, 4.60000000e+01, 4.14980051e-02, 4.14980051e-02],
                                   [4.70000000

In [ ]:
print('média',result_final.mean())
print('mediana',result_final.median())

média Timeframe
0   -0.641008
1   -0.665225
2   -0.126683
3    0.103369
4    1.271620
dtype: float64
mediana Timeframe
0   -1.000000
1   -1.000000
2   -0.103956
3    0.109934
4    0.306170
dtype: float64


In [ ]:
print('média',result_final_DT.mean())
print('mediana',result_final_DT.median())

média Timeframe
0   -0.641008
1   -0.665225
2   -0.126683
3   -0.059027
4    0.279479
dtype: float64
mediana Timeframe
0   -1.000000
1   -1.000000
2   -0.103956
3   -0.073540
4    0.133594
dtype: float64
